In [1]:
####### YOLOv8 포즈 추정 모델 불러오기
from ultralytics import YOLO


model = YOLO("../models/yolov8m-pose.pt")

In [2]:
####### 비디오 파일 불러오기
import cv2

# 비디오파일 or 카메라장치 불러오기
capture = cv2.VideoCapture("../datasets/woman.mp4")
while cv2.waitKey(10) < 0:
    # 비디오가 끝에 도달하면 처음으로 되돌리는 if문
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
    # 다음 프레임 읽어 화면에 표시
    ret, frame = capture.read()
    cv2.imshow("VideoFrame", frame)

# 비디오 읽기 클래스 닫고 매모리 해제
capture.release()
# 창을 제거
cv2.destroyAllWindows()

In [3]:
####### 모델 추론
import torch


def predict(frame, iou=0.7, conf=0.25):
    results = model(
        source  = frame,
        # device  = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu",
        device  = "cuda" if torch.cuda.is_available() else "cpu",
        iou     = 0.7,      # 중복 경계 상자 제거하는 임계값
        conf    = 0.25,     # 클래스 점수 임계값
        verbose = False,    # 모델 수행 시 출력되는 정보 표시 여부 설정
    )
    result = results[0]
    return result

In [4]:
####### 경계 상자 시각화
def draw_boxes(result, frame):
    for boxes in result.boxes:
        x1, y1, x2, y2, score, classes = boxes.data.squeeze().cpu().numpy()
        # 이미지, (x1,y1), (x2,y2), 색상, 선 두께
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
    return frame

In [5]:
####### 경계 상자 추론 및 시각화 적용
import cv2


capture = cv2.VideoCapture("../datasets/woman.mp4")
while cv2.waitKey(10) < 0:
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
    ret, frame = capture.read()
    # 추론 결과 받아 시각화
    result = predict(frame)
    frame  = draw_boxes(result, frame)
    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()

In [6]:
####### 키 포인트 시각화
from ultralytics.utils.plotting import Annotator


def draw_keypoints(result, frame):
    annotator = Annotator(frame, line_width=1)
    for kps in result.keypoints:    # 키포인트는 사람의 수만큼 존재하므로 반복문
        kps = kps.data.squeeze()
        annotator.kpts(kps)
        
        # 키포인트 = [x, y, conf]
        nkps = kps.cpu().numpy()
        # nkps[:,2] = 1
        # annotator.kpts(nkps)
        for idx, (x, y, score) in enumerate(nkps):
            if score > 0.5: # 정확도가 0.5 이상인 것만 시각화
                cv2.circle(frame, (int(x), int(y)), 3, (0, 0, 255), cv2.FILLED)
                cv2.putText(frame, str(idx), (int(x), int(y)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
        
    return frame

In [ ]:
# 키 포인트 추론 및 시각화 적용
import cv2


capture = cv2.VideoCapture("../datasets/woman.mp4")
while cv2.waitKey(10) < 0:
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
    ret, frame = capture.read()
    result = predict(frame)
    frame  = draw_boxes(result, frame)
    frame  = draw_keypoints(result, frame)
    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()